In [12]:
import sys
import logging

import numpy as np
import scipy as sp
import sklearn
import statsmodels.api as sm
from statsmodels.formula.api import ols

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from lrann.datasets import DataLoader, random_train_test_split
from lrann.estimators import ImplicitEst
from lrann.models import BilinearNet, DeepNet
from lrann.evaluations import mrr_score, precision_recall_score
from lrann.utils import is_cuda_available

In [14]:
data = DataLoader().load_movielens('100k-old')

In [15]:
train, test = random_train_test_split(data)

## Estimator using a low-rank approximation model

In [16]:
lra_model = BilinearNet(data.n_users, data.n_items, embedding_dim=32, sparse=False)
lra_est = ImplicitEst(model=lra_model, 
                      n_iter=5, 
                      use_cuda=is_cuda_available())
lra_est.fit(train, verbose=True)

Epoch 0: loss 0.2793229014158249
Epoch 1: loss 0.16784010788202286
Epoch 2: loss 0.15143999789953233
Epoch 3: loss 0.14298499312400817
Epoch 4: loss 0.1384033023416996


In [17]:
prec, recall = precision_recall_score(lra_est, test)
prec.mean(), recall.mean()

(0.11686108165429482, 0.09241441314180279)

In [18]:
mrr_score(lra_est, train).mean(), mrr_score(lra_est, test).mean()

(0.04999118996946312, 0.04313170839048392)

## Estimator using a deep neural model

In [19]:
nn_model = DeepNet(data.n_users, data.n_items, embedding_dim=32, sparse=False)
nn_est = ImplicitEst(model=nn_model, 
                     n_iter=5, 
                     use_cuda=is_cuda_available())
nn_est.fit(train, verbose=True)

Epoch 0: loss 0.23397547523975373
Epoch 1: loss 0.21132075523138047
Epoch 2: loss 0.20922331668138505
Epoch 3: loss 0.20933768365383149
Epoch 4: loss 0.20521306723356247


In [20]:
prec, recall = precision_recall_score(nn_est, test)
prec.mean(), recall.mean()

(0.09056203605514317, 0.06958518641751826)

In [21]:
mrr_score(lra_est, train).mean(), mrr_score(nn_est, test).mean()

(0.04999118996946312, 0.030031285683716713)